In [1]:
#import your packages and modules
import numpy as np
import pandas as pd
import sklearn

In [2]:
train_datalink = 'https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz'

df = pd.read_csv(train_datalink, nrows=50000) #download the training data and keep only the first 50,000 rows

In [3]:
df.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target_kazutsugi
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.5,0.25,0.25,0.75,0.50,1.00,0.5,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.5,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.75
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.25,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.0,0.25,1.00,0.50,1.00,1.00,0.5,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.5,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.25,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.5,1.00,0.00,1.00,1.00,0.50,1.0,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.0,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.00
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.0,0.75,1.00,0.75,1.00,0.75,1.0,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.5,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.00
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.5,1.00,0.50,0.75,0.25,0.50,0.0,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.5,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


We have an additional data step to consider before we define X and y.

Numerai's training data consists of eras, which are time-ordered groupings of data.

How many eras does our subsample contain?

We can use pandas to identify the unique observations in a column.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html

We've done similar things in the past, but this may refresh your memory.

df.era.nunique()

In [4]:
#identify how many eras are contained in the subsample
df.era.nunique()

17

We need to change the 'era' column to contain only numbers so that we can easily split the data into a train and test set.

In [5]:
df.era.unique()

array(['era1', 'era2', 'era3', 'era4', 'era5', 'era6', 'era7', 'era8',
       'era9', 'era10', 'era11', 'era12', 'era13', 'era14', 'era15',
       'era16', 'era17'], dtype=object)

In [6]:
df['era'] = df.loc[:, 'era'].str[3:].astype('int32')

#what this line does:

#1 - over-write the existing 'era' column: df['era'] =
#2 - select only the column named 'era' and all of the observations in that column: df.loc[:, 'era']
#3 - for each observation in the selected column, remove the first 3 characters: .str[3:]
#4 - convert the data type to an integer32 which uses less memory: .astype('int32')

#this is called method chaining

This is a small subset of the 120 total training eras available.

We need to split the data into training and testing sets.

We need to manually partition the data into train and test sets. Let's set aside 8 eras for testing.

`df_train = df[df.era < 10].copy()`

In [7]:
df_train = df[df.era < 10].copy() #nothing is printed. if you want to see the results of this operation, then add a new line and print the first 5 rows of df_train.

In [8]:
df_train.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target_kazutsugi
0,n000315175b67977,1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.5,0.25,0.25,0.75,0.50,1.00,0.5,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.5,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.75
1,n0014af834a96cdd,1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.25,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.0,0.25,1.00,0.50,1.00,1.00,0.5,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.5,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.25,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.5,1.00,0.00,1.00,1.00,0.50,1.0,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.0,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.00
3,n0034e4143f22a13,1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.0,0.75,1.00,0.75,1.00,0.75,1.0,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.5,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.00
4,n00679d1a636062f,1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.5,1.00,0.50,0.75,0.25,0.50,0.0,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.5,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [9]:
df_train.era.unique() #this shows you the actual eras that were selected in the code to define df_train

array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [10]:
#now define your test set.
#HINT: above, you defined df_train. define df_test, and convert the operation above to something that would select all eras other than what was already selected.
#Verify you got the right answer by following the code directly above this cell but change df_train to df_test

df_test = df[df.era > 9].copy() #this code tells python to select all eras that are greater than 9, whereas the train set was defined as all eras less than 10. This is a simple logic test.

In [11]:
#Now we have two dataframes, and need to define X, y and era twice. Let's do that now.

X_train = df_train.iloc[:, 3:313].values

X_test = df_test.iloc[:, 3:313].values

y_train = df_train.target_kazutsugi.values

y_test = df_test.target_kazutsugi.values

era_train = df_train.era.values

era_test = df_test.era.values

I mentioned that cross validation helps to evaluate models and to mitigate the effects of overfitting.

We need to further partition our data into cross validation folds.

Thankfully, Scikit-Learn has functions to do that for us. Let's take a look at the Cross Validation functions in scikit-learn.

https://scikit-learn.org/stable/modules/cross_validation.html

We know already that our data has groups, so we should prefer cross validators that account for groups.

https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators-for-grouped-data

Within this section, there are a couple of strategies.

We can visualize the methodology:

![alt text](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_005.png)
 
Please note that there are four different cross validation iterators for grouped data.

I want you to read about each one and think about your future production algorithm. Which one do you want to rely on to generate the best possible inference from the training data?

I will demonstrate how to use a cross-validation iterator using the first option. This may not be appropriate for your model!

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html#sklearn.model_selection.GroupKFold

This time, we'll need to choose parameters and not rely on the default settings. Visit the link above and read the Parameters section.

 
You'll notice that the default number of splits is 5. We happen to have 9 eras, which splits evenly into 3 groups.

The warning in the parameter section tells you the default setting changed in version 0.22 from 3 to 5. This is why it is important to define the settings for production algorithms which may require stability over several years.

While we could rely on the default settings for now, let's instead use n_splits=3

![alt text](https://i.postimg.cc/WpqxTCv7/image.png)

In [12]:
#import the model_selection module

from sklearn import model_selection

#define your cross-validation iterator

CV = model_selection.GroupKFold(n_splits = 3)

Now that we've defined our splitter, we have to use it to generate our cross-validation partitions.

I demonstrated last lecture that you can access the functions of a model (or iterator) by viewing the Methods section.

![alt text](https://i.postimg.cc/mZc3qGsT/Capture.jpg)
 
We can access the methods using '.' on CV:

![alt text](https://i.postimg.cc/BbZrSQc7/method-parameters.jpg)
 
What happens behind the scenes is that the data is sliced into further train and test sets, and the output from using .split() is an array which models in the next step can use automatically to train and test on the different partitions. This isn't easy to visualize, but the example code in the link gives a pretty good explanation of the output. You are given train and test sets which identify the rows that belong to each grouping.

In [13]:
#We can now define a new variable which stores these row identifiers for algorithms to use.

grp = list(CV.split(X = X_train, y = y_train,  groups = era_train))

#what this line does is:

#1 - define a new variable: grp = 
#2 - use the python 'list' function: list(
#3 - within the list function, we use the .split() method on our group k-fold iterator, which we defined as CV: CV.split(
#4 - within the .split() method, we defined the required parameters and closed the .split parenthesis: X = X_train, y = y_train, groups = era_train)
#5 - we enclosed the list function: )

#this is called nested function.

You may have noticed that many models include parameters which can be changed from the default value. Doing so can increase the model's performance, and also lead to overfitting. The benefit of cross validation, and models with built-in cross validation optimization, is that you don't have to do the work of determining what value is the most performant.

Let's use Ridge Regression as an example.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV

![alt text](https://i.postimg.cc/sxRp7v0K/ridge-CV.jpg)

The model may improve by changing the alpha parameter, but we don't know which value to choose in advance. So we should just try a whole bunch and let the math wizards who wrote this code figure it out!

We can define the search space ourselves by creating a new variable and listing the values to evaluate explicitly like so:

`alphas = (0.1, 0.5, 1.0, 10.0)`

Since we are altering the default settings of RidgeCV then we have to modify those terms within the parenthesis:

`RidgeCV(alphas = alphas, cv = grp)`

above, we deviate from the default parameters for alphas and cv, which we've defined as variables ourselves above. Your python environment has stored those variables in memory, and RidgeCV will "call" those variables as part of the fit function automatically. Let's put this to work and determine which alpha gives the best in-sample performance.

In [14]:
from sklearn import linear_model
alphas = (0.1, 0.5, 1.0, 10.0)
REG1 = linear_model.RidgeCV(alphas = alphas, cv = grp)
REG1.fit(X_train, y_train)
#you can just ignore the output that is generated below once the code finishes

RidgeCV(alphas=array([ 0.1,  0.5,  1. , 10. ]),
        cv=[(array([ 2408,  2409,  2410, ..., 21341, 21342, 21343]),
             array([    0,     1,     2, ..., 24512, 24513, 24514])),
            (array([    0,     1,     2, ..., 24512, 24513, 24514]),
             array([ 4779,  4780,  4781, ..., 18323, 18324, 18325])),
            (array([    0,     1,     2, ..., 24512, 24513, 24514]),
             array([ 2408,  2409,  2410, ..., 21341, 21342, 21343]))],
        fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
        store_cv_values=False)

We can access stored values:

![alt text](https://i.postimg.cc/LX4SXTmV/access.jpg)

In [15]:
REG1.alpha_ #This line prints the alpha value that gave the model the best fit

10.0

In [16]:
REG1.score(X_train, y_train) #scoring the model gives you the R-sq of the model using the best alpha parameter

0.02141160846563162

This may be insufficent for your analysis. What if you wanted to see the performance of each term?

We can use a model selection tool called Grid Search CV.

Let's turn this up to 11. 

![alt text](https://i.postimg.cc/SxjDyz0z/to-11.jpg)

https://scikit-learn.org/stable/modules/grid_search.html

Grid search allows us to "span the space" and determine which parameter is best and also to give you the performance in a table. This is a very powerful tool!

Let's keep using Ridge, but now we will use the non-CV version.

In [17]:
REG2 = linear_model.Ridge()

#we also have to create a python dictionary for grid search to use:

params1 = {'alpha': [0.1, 0.3, 0.5, 0.8, 1.0, 2.0, 5.0, 10.0]}

#what this line does is:

#a dictionary has a dimension n,j (think columns and rows, although that isn't quite right)

#1 - define a new variable: params = 
#2 - create a python 'dictionary': {
#3 - define the name of the nth item as: 'alpha':
#4 - the jth item is a list: [
#5 - inside the jth item list is: 0.1, 0.3, 0.5, 0.8, 1.0, 2.0, 5.0, 10.0]
#6 - close the dictionary: }

#we created a dictionary that contains a list: https://docs.python.org/3/tutorial/datastructures.html#dictionaries

In [18]:
GS1 = model_selection.GridSearchCV(estimator = REG2, param_grid = params1, cv = grp, return_train_score = True)
GS1.fit(X_train, y_train)
#you can ignore the output that is generated below after the code finishes

GridSearchCV(cv=[(array([ 2408,  2409,  2410, ..., 21341, 21342, 21343]),
                  array([    0,     1,     2, ..., 24512, 24513, 24514])),
                 (array([    0,     1,     2, ..., 24512, 24513, 24514]),
                  array([ 4779,  4780,  4781, ..., 18323, 18324, 18325])),
                 (array([    0,     1,     2, ..., 24512, 24513, 24514]),
                  array([ 2408,  2409,  2410, ..., 21341, 21342, 21343]))],
             error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.1, 0.3, 0.5, 0.8, 1.0, 2.0, 5.0, 10.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [19]:
scores1 = pd.DataFrame(GS1.cv_results_); scores1

#if we type GS.cv_results_ then we are given a dictionary. We can use Pandas to convert a dictionary to a dataframe using method chaining.

#the code above defines a new variable called scores1, and we use the DataFrames() function from Pandas to interpret the GS1.cv_results_ method's output as a dataframe.
#we then used the shortcut method: ; scores1 to print the table below.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.135278,0.001205,0.004932,0.000064,0.1,{'alpha': 0.1},-0.017317,-0.015845,-0.013093,-0.015419,0.001751,8,0.029243,0.029431,0.027446,0.028707,0.000895
1,0.139140,0.004838,0.004924,0.000103,0.3,{'alpha': 0.3},-0.017275,-0.015803,-0.013057,-0.015378,0.001748,7,0.029243,0.029431,0.027446,0.028706,0.000895
2,0.136483,0.000336,0.005144,0.000408,0.5,{'alpha': 0.5},-0.017232,-0.015760,-0.013021,-0.015338,0.001745,6,0.029243,0.029431,0.027445,0.028706,0.000895
3,0.137973,0.002653,0.004864,0.000160,0.8,{'alpha': 0.8},-0.017169,-0.015698,-0.012967,-0.015278,0.001741,5,0.029242,0.029430,0.027445,0.028706,0.000895
4,0.137296,0.002691,0.004802,0.000059,1,{'alpha': 1.0},-0.017127,-0.015656,-0.012932,-0.015238,0.001738,4,0.029242,0.029430,0.027445,0.028705,0.000895
5,0.138746,0.005615,0.004806,0.000041,2,{'alpha': 2.0},-0.016923,-0.015454,-0.012758,-0.015045,0.001725,3,0.029239,0.029426,0.027442,0.028702,0.000895
6,0.147278,0.016978,0.004810,0.000105,5,{'alpha': 5.0},-0.016347,-0.014890,-0.012267,-0.014502,0.001688,2,0.029220,0.029403,0.027422,0.028681,0.000894
7,0.136225,0.001423,0.004742,0.000087,10,{'alpha': 10.0},-0.015497,-0.014068,-0.011542,-0.013702,0.001636,1,0.029159,0.029334,0.027362,0.028618,0.000891


![alt text](https://i.postimg.cc/PfQjD0CQ/winning-jpg.jpg)

WE HAVE DATA!!!



In [20]:
#which model is the best?

GS1.best_estimator_

#the output below contains the parameters used in the best model. You could copy this and define it as a new variable for production if you wanted to.
#alpha=10.0, copy_X=True, fit_intercept=True, max_iter=None,
#      normalize=False, random_state=None, solver='auto', tol=0.001

Ridge(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
GS1.best_score_

#this is the score given by the model above. Our score is negative! We've actually overfit, and the model is worse than just guessing randomly.
#However, this statement only holds for eras 1-9; perhaps the model would perform better with more data.
#I leave that to you to evaluate with a computer that has more resources available.

-0.013702340474596175

In [22]:
#this code will take almost 10 minutes to run. Please be patient!

#####################################################################################################################################################################################################################################

#What if a model has more than one parameter which can be optimized? Grid Search CV can handle that scenario as well. Here's how we handle more than one parameter optimization.
#Let's find the best combination of l1_ratio and tol in the ElasticNet model:

#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

params2 = {'l1_ratio': [0.0, 0.25, 0.33, 0.5, 0.66, 0.75, 1], 'tol': [0.01, 0.001, 0.0001, 0.00001]}

#what will happen in the background is that grid search will use l1_ratio of 0.0 and evaluate performance for each value of 'tol', and so on for every value of l1_ratio provided. This takes a long time with just 9 eras; imagine how long it would take to do this with all 120 eras! While trying a lot of parameters is a good idea, you've got to balance efficiency with your desire to find the best settings for the model. This is an art, not a science.

#You'll see some warning messages about convergence. You can ignore them for now.

REG3 = linear_model.ElasticNet(max_iter = 3000) #note I changed a default parameter! I am using a maximum of 3,000 training iterations to find the best fit for each run of the model.

GS2 = model_selection.GridSearchCV(estimator = REG3, param_grid = params2, cv = grp, return_train_score = True)

GS2.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1018.8279876603805, tolerance: 20.441874656865707
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1019.3122907399952, tolerance: 20.45125
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1010.219370420332, tolerance: 20.265623115500564
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1018.827987660380

GridSearchCV(cv=[(array([ 2408,  2409,  2410, ..., 21341, 21342, 21343]),
                  array([    0,     1,     2, ..., 24512, 24513, 24514])),
                 (array([    0,     1,     2, ..., 24512, 24513, 24514]),
                  array([ 4779,  4780,  4781, ..., 18323, 18324, 18325])),
                 (array([    0,     1,     2, ..., 24512, 24513, 24514]),
                  array([ 2408,  2409,  2410, ..., 21341, 21342, 21343]))],
             error_score=nan,
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=3000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'l1_ratio': [0.0, 0.25, 0.33, 0.5, 0.66, 0.75, 1],
                         'tol': [0.01,

In [23]:
scores2 = pd.DataFrame(GS2.cv_results_); scores2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_tol,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,31.186899,0.500659,0.004953,0.000119,0,0.01,"{'l1_ratio': 0.0, 'tol': 0.01}",7.311680e-04,1.456584e-03,1.745255e-03,1.311002e-03,4.266058e-04,1,0.005508,0.005367,0.005165,0.005347,0.000141
1,30.867608,0.244312,0.005089,0.000281,0,0.001,"{'l1_ratio': 0.0, 'tol': 0.001}",7.311680e-04,1.456584e-03,1.745255e-03,1.311002e-03,4.266058e-04,1,0.005508,0.005367,0.005165,0.005347,0.000141
2,30.899640,0.250195,0.004926,0.000096,0,0.0001,"{'l1_ratio': 0.0, 'tol': 0.0001}",7.311680e-04,1.456584e-03,1.745255e-03,1.311002e-03,4.266058e-04,1,0.005508,0.005367,0.005165,0.005347,0.000141
3,31.161338,0.187209,0.005089,0.000015,0,1e-05,"{'l1_ratio': 0.0, 'tol': 1e-05}",7.311680e-04,1.456584e-03,1.745255e-03,1.311002e-03,4.266058e-04,1,0.005508,0.005367,0.005165,0.005347,0.000141
4,0.098674,0.002117,0.005056,0.000032,0.25,0.01,"{'l1_ratio': 0.25, 'tol': 0.01}",-2.002597e-09,-1.897815e-07,-2.266571e-07,-1.394804e-07,9.837025e-08,5,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.100142,0.002290,0.005297,0.000212,0.25,0.001,"{'l1_ratio': 0.25, 'tol': 0.001}",-2.002597e-09,-1.897815e-07,-2.266571e-07,-1.394804e-07,9.837025e-08,5,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.096334,0.000646,0.005005,0.000059,0.25,0.0001,"{'l1_ratio': 0.25, 'tol': 0.0001}",-2.002597e-09,-1.897815e-07,-2.266571e-07,-1.394804e-07,9.837025e-08,5,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.096384,0.000552,0.005105,0.000046,0.25,1e-05,"{'l1_ratio': 0.25, 'tol': 1e-05}",-2.002597e-09,-1.897815e-07,-2.266571e-07,-1.394804e-07,9.837025e-08,5,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.110115,0.010452,0.005440,0.000276,0.33,0.01,"{'l1_ratio': 0.33, 'tol': 0.01}",-2.002597e-09,-1.897815e-07,-2.266571e-07,-1.394804e-07,9.837025e-08,5,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.097500,0.000558,0.005253,0.000156,0.33,0.001,"{'l1_ratio': 0.33, 'tol': 0.001}",-2.002597e-09,-1.897815e-07,-2.266571e-07,-1.394804e-07,9.837025e-08,5,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
#determine which model was the best
GS2.best_estimator_

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.0,
           max_iter=3000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.01, warm_start=False)

In [25]:
#determine the score of the best model
GS2.best_score_

0.0013110024971313787

In [ ]:
#Now that we've determined which model is the best, we can evaluate the performance of the model on the out-of-sample test set.

#First, we have to re-train the model on the training data.

REG4 = GS2.best_estimator_ #we can easily define the best model without writing very much, since GridSearchCV allows us to access the best model directly
REG4.fit(X_train, y_train)

In [ ]:
#Let's double check the in-sample performance
REG4.score(X_train, y_train)

Numerai scores your predictions based on correlation with the target variable. Until this point, we've only considered the performance of the model based on the R-sq. Numerai provides the scoring code.

We can define a python function (https://docs.python.org/3/tutorial/controlflow.html#defining-functions) to calculate the correlation score:

In [ ]:
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

In [ ]:
#Let's generate some predictions. Since we've fit the model to the training data, we can generate predictions based on the test set.
#We want the highest possible correlation score with the understanding that a very high score may be a sign that the model has overfit.

preds1 = REG4.predict(X_test)

In [ ]:
#Now let's calculate the out-of-sample correlation using Numerai's scoring function and print the score:

OOS_score1 = correlation_score(y_test, preds1); OOS_score1

![alt text](https://i.postimg.cc/7LR1Yx8P/thumbs-up.jpg)

5.7% correlation is pretty good! There are some caveats to remember. You trained the model on 9 out of 120 eras, and evaluated the performance based on 8 of the 111 eras remaining. We know that there is a time-series component to the data, so perhaps eras 1-9 are similar to eras 10-17. We still don't know if the model is any good, but we did take a scientific approach to the evaluation.

Because we held out data that the model has never seen, we can infer that if live stock market data is similar to eras 10-17, then we should expect our models to generate a 5.7% correlation, on average. This is highly unlikely in practice!

We have another problem. We trained the model on eras 1-9, so the assumption is that each era is nearly identical in composition. We know this to be false as well. The more likely case is that some of the eras are more similar than others. What if we used grid search to identify the best parameters to use on eras 1-3, 4-6, and 7-9? What if we fit those best models to 1-3, 4-6, and 7-9 and then generated predictions on the test set for each of them and averaged the predictions across the three models? That would be a much better approach, wouldn't it?

Here's the thing: Even if you did average across groups like that, there's no guarantee that eras 1-3 are sufficiently similar to eachother but sufficiently different from 4-6 and 7-9 to improve the forecast. If 1-3, 4-6, and 7-9 are actually similar, then averaging the 3 prediction sets would yield an answer that was no different from any one of the 3 prediction sets.

And another layer of complexity exists that we haven't even addressed yet. Some of the features are highly correlated. If your model does not address this multicollinearity then the model is likely to overfit those features, causing bad inference.

ASSIGNMENT: Use the scikit-learn website to identify three models (in this case, a model is a sequence of tasks that eventually generate predictions) which work with our dataset (regression algorithms) that:

-either automatically handle multicollinearity or offer some way to mitigate multicollinearity

-mitigate problems with training on the entire dataset, such as averaging across a number of models (hint: ensemble methods)

-or a combination of a feature selection tool followed by an algorithm which will work with our dataset

You must also identify which parameters can be optimized.

![alt text](https://i.postimg.cc/V6D8bHgN/supervised-learning.jpg)
 
Below is an example of a correct answer. Note that just because there are three models within the answer, that doesn't mean that you are finished. You still must develop 2 additional models. Perhaps you find an algorithm that does everything you want and doesn't require a long answer such as below. That's fine, but you should justify that decision in the description. This is an opportunity for you to think about your production model and to show me some of your ideas, and also that you understand the concepts we've discussed. If you find one you like but are struggling to understand how to set it up properly, then please join Numerai's Rocket Chat in the #newusers channel and I can help you.

In [ ]:
#MODEL 1: Voting Regressor

#LINK: https://scikit-learn.org/stable/modules/ensemble.html#voting-regressor

#Description of methodology and how the model helps to give better performance:

#Voting Regressor requires that several models be defined which will each be fit on the training data.
#The voting regressor then averages the predictions made by each model, which balances out the individual weaknesses of the models.
#This will help to reduce overfitting, but only if the models are sufficienty different.

#Import the relevant modules, define the model(s), identify parameters to be optimized:

from sklearn import ensemble
from sklearn import kernel_ridge
from sklearn import neural_network
VR1 = kernel_ridge.KernelRidge() #kernel ridge allows for optimization of the alpha parameter.
VR2 = neural_network.MLPRegressor() #MLP regressor allows for optimization of many parameters: the activation function, the solver, alpha, batch_size, learning_rate_init, max_iter, tol, and many other parameters. So many, that if I choose to use this model in production, then I will ask the Professor for help before I try to use this in practice!
VR3 = linear_model.ElasticNet() #ElasticNet allows for optimization of these parameters: alpha, l1_ratio, tol

#the example code given for Voting Regressor tells me that I have to provide a list of estimators within the parenthesis, so I follow the example like this:

META = ensemble.VotingRegressor(estimators=[('VR1', VR1), ('VR2', VR2), ('VR3', VR3)])

#Write the correct code to fit your model:

META.fit(X_train, y_train)

#Write the code to generate and store predictions from your fitted model:

#you can name this whatever you want, here are some examples:

#PREDS1
#y_pred
#predictions

VR_pred = META.predict(X_test)

#Write the code to score the out-of-sample predictions using the correlation function and print the correlation score:

#you can name this whatever you want

OOS_VR_score = correlation_score(y_test, VR_pred); OOS_VR_score

In [ ]:
#MODEL 1: Elastic Net

#LINK: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet

#Description of methodology and how the model helps to give better performance:

#Elastic-net is useful when there are multiple features which are correlated with one another. Lasso is likely to pick one of these at random, while elastic-net is likely to pick both.

#Import the relevant modules, define the models, identify parameters to be optimized:

M1 = linear_model.ElasticNet() # parameters that can be optimized are: alpha, l1_ratio, tol

#Write the correct code to fit your model:

M1.fit(X_train, y_train)

#Write the code to generate and store predictions from your fitted model:

OOS_M1 = M1.predict(X_test)

#Write the code to score the out-of-sample predictions using the correlation function and print the correlation score:

OOS_M1_score = correlation_score(y_test, OOS_M1); OOS_M1_score

In [ ]:
#MODEL 2: 

#LINK:

#Description of methodology and how the model helps to give better performance:

#Import the relevant modules, define the models, identify parameters to be optimized:
#parameters that can be optimized are: base_estimator, n_estimators, learning_rate, loss, random_state

#Write the correct code to fit your model:

#Write the code to generate and store predictions from your fitted model:

#Write the code to score the out-of-sample predictions using the correlation function and print the correlation score:


In [ ]:
#MODEL 3: 

#LINK:

#Description of methodology and how the model helps to give better performance:

#Import the relevant modules, define the models, identify parameters to be optimized:

#Write the correct code to fit your model:

#Write the code to generate and store predictions from your fitted model:

#Write the code to score the out-of-sample predictions using the correlation function and print the correlation score: